# 🚀 YOLOv8 Training - Third Iteration
**Enhanced Training with Optimized Hyperparameters**

## 📊 Previous Results:
- **Training 1:** Baseline model
- **Training 2 (Fine-tune):** Precision 0.793, Recall 0.679, mAP@0.5: 0.775

## 🎯 Target for Training 3:
- **Precision:** > 0.85
- **Recall:** > 0.75
- **mAP@0.5:** > 0.85
- **mAP@0.5-0.95:** > 0.60


## 🔧 Setup Environment


In [ ]:
import os
import sys
from ultralytics import YOLO
import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Check GPU availability
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print(f"PyTorch version: {torch.__version__}")
print(f"Ultralytics version: {YOLO.__version__}")


## 📁 Dataset Configuration


In [ ]:
# Dataset paths
data_yaml = "G:/Yolov8n/datasets/potholes_raw/data.yaml"
base_model = "runs/detect/yolov8n-potholes-ft/weights/best.pt"

# Verify paths exist
print(f"Data YAML exists: {os.path.exists(data_yaml)}")
print(f"Base model exists: {os.path.exists(base_model)}")

# Check dataset structure
if os.path.exists(data_yaml):
    with open(data_yaml, 'r') as f:
        print("\nDataset configuration:")
        print(f.read())


## ⚠️ Important: Run Training First!

**Before running the validation and test sections below, make sure to:**
1. Run the training configuration cell (cell above)
2. Run the training cell to completion
3. Then run the validation and test cells


## 🎯 Training Configuration - Third Iteration


In [ ]:
# Training parameters for third iteration
training_config = {
    'data': data_yaml,
    'model': base_model,  # Start from fine-tuned model
    'epochs': 100,  # Optimal training epochs
    'imgsz': 640,
    'batch': 8,
    'name': 'yolov8n-potholes-v3',
    
    # Optimized hyperparameters
    'lr0': 0.005,  # Lower learning rate for fine-tuning
    'lrf': 0.005,  # Final learning rate
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 5,  # Increased warmup
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,
    'patience': 20,  # Early stopping patience for optimal training
    
    # Enhanced data augmentation
    'hsv_h': 0.02,  # Increased from 0.015
    'hsv_s': 0.8,   # Increased from 0.7
    'hsv_v': 0.5,   # Increased from 0.4
    'degrees': 5.0, # Added rotation
    'translate': 0.15,  # Increased from 0.1
    'scale': 0.6,   # Increased from 0.5
    'shear': 2.0,   # Added shear
    'perspective': 0.1,  # Added perspective
    'flipud': 0.0,
    'fliplr': 0.5,
    'mosaic': 1.0,
    'mixup': 0.1,   # Added mixup
    
    # Training settings
    'save': True,
    'save_period': 20,  # Save checkpoint every 20 epochs
    'cache': False,  # Disable caching for memory efficiency
    'workers': 4,   # Number of workers
    'project': 'runs/detect',
    'exist_ok': True,  # Overwrite existing runs
}

print("Training Configuration:")
for key, value in training_config.items():
    print(f"  {key}: {value}")


## 🚀 Start Training - Third Iteration


In [ ]:
# Load model
model = YOLO(training_config['model'])

print(f"Starting training with model: {training_config['model']}")
print(f"Target epochs: {training_config['epochs']}")
print(f"Learning rate: {training_config['lr0']}")
print(f"Batch size: {training_config['batch']}")
print(f"Image size: {training_config['imgsz']}")

# Start training
results = model.train(
    data=training_config['data'],
    epochs=training_config['epochs'],
    imgsz=training_config['imgsz'],
    batch=training_config['batch'],
    name=training_config['name'],
    lr0=training_config['lr0'],
    lrf=training_config['lrf'],
    momentum=training_config['momentum'],
    weight_decay=training_config['weight_decay'],
    warmup_epochs=training_config['warmup_epochs'],
    warmup_momentum=training_config['warmup_momentum'],
    warmup_bias_lr=training_config['warmup_bias_lr'],
    patience=training_config['patience'],
    hsv_h=training_config['hsv_h'],
    hsv_s=training_config['hsv_s'],
    hsv_v=training_config['hsv_v'],
    degrees=training_config['degrees'],
    translate=training_config['translate'],
    scale=training_config['scale'],
    shear=training_config['shear'],
    perspective=training_config['perspective'],
    flipud=training_config['flipud'],
    fliplr=training_config['fliplr'],
    mosaic=training_config['mosaic'],
    mixup=training_config['mixup'],
    save=training_config['save'],
    save_period=training_config['save_period'],
    cache=training_config['cache'],
    workers=training_config['workers'],
    project=training_config['project'],
    exist_ok=training_config['exist_ok']
)

print("\n✅ Training completed!")
print(f"Results saved to: runs/detect/{training_config['name']}")


## 📊 Model Validation & Testing


In [ ]:
# Load the trained model for validation
trained_model_path = f"runs/detect/{training_config['name']}/weights/best.pt"
model = YOLO(trained_model_path)

print(f"Loaded trained model: {trained_model_path}")
print(f"Model exists: {os.path.exists(trained_model_path)}")

# Validate the model
print("\n🔍 Running validation...")
val_results = model.val(
    data=training_config['data'],
    imgsz=training_config['imgsz'],
    batch=training_config['batch'],
    save_json=True,
    save_hybrid=True,
    plots=True
)

print("\n📈 Validation Results:")
print(f"mAP@0.5: {val_results.box.map50:.4f}")
print(f"mAP@0.5-0.95: {val_results.box.map:.4f}")
print(f"Precision: {val_results.box.mp:.4f}")
print(f"Recall: {val_results.box.mr:.4f}")
print(f"F1-Score: {val_results.box.f1:.4f}")

# Check if targets are met
target_precision = 0.85
target_recall = 0.75
target_map50 = 0.85
target_map = 0.60

print(f"\n🎯 Target Achievement:")
print(f"Precision: {val_results.box.mp:.4f} / {target_precision} {'✅' if val_results.box.mp >= target_precision else '❌'}")
print(f"Recall: {val_results.box.mr:.4f} / {target_recall} {'✅' if val_results.box.mr >= target_recall else '❌'}")
print(f"mAP@0.5: {val_results.box.map50:.4f} / {target_map50} {'✅' if val_results.box.map50 >= target_map50 else '❌'}")
print(f"mAP@0.5-0.95: {val_results.box.map:.4f} / {target_map} {'✅' if val_results.box.map >= target_map else '❌'}")


## 🧪 Test Dataset Inference


In [ ]:
# Test dataset inference
test_images_path = "datasets/potholes_raw/test/images"
test_output_path = f"runs/detect/{training_config['name']}-test-predict"

print(f"🔍 Running inference on test dataset...")
print(f"Test images path: {test_images_path}")
print(f"Output path: {test_output_path}")

# Run inference on test dataset
test_results = model.predict(
    source=test_images_path,
    imgsz=training_config['imgsz'],
    save=True,
    save_txt=True,
    save_conf=True,
    conf=0.25,  # Confidence threshold
    iou=0.45,   # IoU threshold for NMS
    project="runs/detect",
    name=f"{training_config['name']}-test-predict",
    exist_ok=True
)

print(f"\n✅ Test inference completed!")
print(f"Results saved to: {test_output_path}")
print(f"Number of test images processed: {len(test_results)}")

# Count predictions
total_predictions = 0
for result in test_results:
    if result.boxes is not None:
        total_predictions += len(result.boxes)
        
print(f"Total potholes detected: {total_predictions}")
print(f"Average detections per image: {total_predictions/len(test_results):.2f}")


## 📊 Performance Analysis & Visualization


In [ ]:
# Performance analysis and visualization
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Create performance summary
performance_summary = {
    'Metric': ['Precision', 'Recall', 'F1-Score', 'mAP@0.5', 'mAP@0.5-0.95'],
    'Value': [
        val_results.box.mp,
        val_results.box.mr, 
        val_results.box.f1,
        val_results.box.map50,
        val_results.box.map
    ],
    'Target': [0.85, 0.75, 0.80, 0.85, 0.60],
    'Achieved': [
        '✅' if val_results.box.mp >= 0.85 else '❌',
        '✅' if val_results.box.mr >= 0.75 else '❌', 
        '✅' if val_results.box.f1 >= 0.80 else '❌',
        '✅' if val_results.box.map50 >= 0.85 else '❌',
        '✅' if val_results.box.map >= 0.60 else '❌'
    ]
}

print("📊 Performance Summary:")
print("=" * 60)
for i, metric in enumerate(performance_summary['Metric']):
    print(f"{metric:12} | {performance_summary['Value'][i]:.4f} | Target: {performance_summary['Target'][i]:.2f} | {performance_summary['Achieved'][i]}")

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Bar chart of metrics
metrics = performance_summary['Metric']
values = performance_summary['Value']
targets = performance_summary['Target']

x = np.arange(len(metrics))
width = 0.35

ax1.bar(x - width/2, values, width, label='Achieved', alpha=0.8, color='skyblue')
ax1.bar(x + width/2, targets, width, label='Target', alpha=0.8, color='lightcoral')

ax1.set_xlabel('Metrics')
ax1.set_ylabel('Score')
ax1.set_title('Model Performance vs Targets')
ax1.set_xticks(x)
ax1.set_xticklabels(metrics, rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Add value labels on bars
for i, (v, t) in enumerate(zip(values, targets)):
    ax1.text(i - width/2, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
    ax1.text(i + width/2, t + 0.01, f'{t:.3f}', ha='center', va='bottom')

# Pie chart of achievement
achieved_count = sum(1 for status in performance_summary['Achieved'] if status == '✅')
total_count = len(performance_summary['Achieved'])
not_achieved_count = total_count - achieved_count

ax2.pie([achieved_count, not_achieved_count], 
        labels=[f'Achieved ({achieved_count})', f'Not Achieved ({not_achieved_count})'],
        colors=['lightgreen', 'lightcoral'],
        autopct='%1.1f%%',
        startangle=90)
ax2.set_title('Target Achievement Status')

plt.tight_layout()
plt.show()

# Save performance report
report_path = f"runs/detect/{training_config['name']}/performance_report.txt"
with open(report_path, 'w') as f:
    f.write("YOLOv8 Pothole Detection - Performance Report\n")
    f.write("=" * 50 + "\n\n")
    f.write(f"Model: {training_config['name']}\n")
    f.write(f"Training Epochs: {training_config['epochs']}\n")
    f.write(f"Learning Rate: {training_config['lr0']}\n")
    f.write(f"Batch Size: {training_config['batch']}\n")
    f.write(f"Image Size: {training_config['imgsz']}\n\n")
    
    f.write("Performance Metrics:\n")
    f.write("-" * 30 + "\n")
    for i, metric in enumerate(performance_summary['Metric']):
        f.write(f"{metric}: {performance_summary['Value'][i]:.4f} (Target: {performance_summary['Target'][i]:.2f}) {performance_summary['Achieved'][i]}\n")
    
    f.write(f"\nTest Dataset Results:\n")
    f.write(f"Total images processed: {len(test_results)}\n")
    f.write(f"Total potholes detected: {total_predictions}\n")
    f.write(f"Average detections per image: {total_predictions/len(test_results):.2f}\n")

print(f"\n📄 Performance report saved to: {report_path}")


## 🎥 Video Inference Test


In [ ]:
# Video inference test
video_path = "datasets/potholes_video/pothole_video.mp4"
video_output_path = f"runs/detect/{training_config['name']}-video-predict"

if os.path.exists(video_path):
    print(f"🎥 Running inference on video: {video_path}")
    
    # Run inference on video
    video_results = model.predict(
        source=video_path,
        imgsz=training_config['imgsz'],
        save=True,
        save_txt=True,
        save_conf=True,
        conf=0.25,
        iou=0.45,
        project="runs/detect",
        name=f"{training_config['name']}-video-predict",
        exist_ok=True
    )
    
    print(f"✅ Video inference completed!")
    print(f"Output video saved to: {video_output_path}")
    
    # Get video info
    import cv2
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps
    cap.release()
    
    print(f"Video info:")
    print(f"  - FPS: {fps:.2f}")
    print(f"  - Total frames: {frame_count}")
    print(f"  - Duration: {duration:.2f} seconds")
    
else:
    print(f"❌ Video file not found: {video_path}")
    print("Skipping video inference test...")


## 📋 Summary & Next Steps


In [ ]:
# Final summary
print("🎯 YOLOv8 Training & Testing Summary")
print("=" * 50)

print(f"\n📊 Model Performance:")
print(f"  • Precision: {val_results.box.mp:.4f}")
print(f"  • Recall: {val_results.box.mr:.4f}")
print(f"  • F1-Score: {val_results.box.f1:.4f}")
print(f"  • mAP@0.5: {val_results.box.map50:.4f}")
print(f"  • mAP@0.5-0.95: {val_results.box.map:.4f}")

print(f"\n📁 Output Files:")
print(f"  • Model weights: runs/detect/{training_config['name']}/weights/")
print(f"  • Validation plots: runs/detect/{training_config['name']}/")
print(f"  • Test predictions: runs/detect/{training_config['name']}-test-predict/")
if os.path.exists(video_path):
    print(f"  • Video predictions: runs/detect/{training_config['name']}-video-predict/")
print(f"  • Performance report: runs/detect/{training_config['name']}/performance_report.txt")

print(f"\n🔍 Test Results:")
print(f"  • Test images processed: {len(test_results)}")
print(f"  • Total potholes detected: {total_predictions}")
print(f"  • Average detections per image: {total_predictions/len(test_results):.2f}")

# Check if all targets achieved
all_targets_met = all(status == '✅' for status in performance_summary['Achieved'])
print(f"\n🎯 Target Achievement: {'✅ ALL TARGETS MET!' if all_targets_met else '❌ Some targets not met'}")
print(f"  • Targets achieved: {sum(1 for status in performance_summary['Achieved'] if status == '✅')}/{len(performance_summary['Achieved'])}")

print(f"\n📈 Next Steps:")
if not all_targets_met:
    print("  • Consider adjusting hyperparameters")
    print("  • Try different data augmentation strategies")
    print("  • Increase training epochs if needed")
    print("  • Consider model architecture modifications")
else:
    print("  • Model is ready for deployment!")
    print("  • Consider real-world testing")
    print("  • Optimize inference speed if needed")

print(f"\n✅ Training and testing pipeline completed successfully!")
